# Batch ClassyFirer

Building on pyMolNetEnhancer (https://pypi.org/project/pyMolNetEnhancer/) this notebook allows to fetch classyfire for large structure tables.
The main mnodification concern the ability to fetch classyfire results for inchi (or smiles) based input. 
The paginated JSON results are fetched using the xxxxx function.

Load libraries required by pyMolNetEnhancer

In [1]:
from pybatchclassyfire import *
import pandas as pd
import os
import csv 
import time
import json
from pandas import json_normalize

Set option for display in pandas

In [2]:
#pd.reset_option('all')

In [3]:
pd.set_option('max_rows', 40)
pd.set_option('max_columns', 40)
#pd.set_option('display.max_colwidth', 300)

## Original data loading

In [4]:
#opendb = pd.read_csv("/Users/pma/Downloads/open_NP_db_sanitized.tsv", sep = "\t")
opendb = pd.read_csv("../../opennaturalproductsdb/outputs/tables/3_curated/curated_structure_min.tsv", 
                     sep = "\t")

Duplicates are droped at the sanitized inchi columns

In [5]:
opendb.drop_duplicates('inchikey_sanitized', inplace = True)

Uncomment if a minimal input is needed for testing purposes

In [6]:
#opendb = opendb.head(5000)

Fetch info on the loaded df

In [7]:
opendb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4253 entries, 0 to 4274
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   structure_translated  4253 non-null   object 
 1   validator_log         4253 non-null   object 
 2   smiles_sanitized      4253 non-null   object 
 3   inchi_sanitized       4253 non-null   object 
 4   inchikey_sanitized    4253 non-null   object 
 5   shortik_sanitized     4253 non-null   object 
 6   formula_sanitized     4253 non-null   object 
 7   exactmass_sanitized   4253 non-null   float64
 8   xlogp_sanitized       4253 non-null   float64
 9   structure_sanitized   4253 non-null   object 
 10  structure_curated     4248 non-null   object 
dtypes: float64(2), object(9)
memory usage: 398.7+ KB


In [8]:
opendb.head()

,structure_translated,validator_log,smiles_sanitized,inchi_sanitized,inchikey_sanitized,shortik_sanitized,formula_sanitized,exactmass_sanitized,xlogp_sanitized,structure_sanitized,structure_curated
0,InChI=1S/C16H26/c1-12(2)14-8-10-15(4)9-6-7-13(...,[],C=C(C)C1CCC2(C)CCC=C(C)[C@@]2(C)C1,InChI=1S/C16H26/c1-12(2)14-8-10-15(4)9-6-7-13(...,IGHNHLWYXDJPIB-UYSNPLJNSA-N,IGHNHLWYXDJPIB,C16H26,218.203451,5.11530,InChI=1S/C16H26/c1-12(2)14-8-10-15(4)9-6-7-13(...,InChI=1S/C16H26/c1-12(2)14-8-10-15(4)9-6-7-13(...
1,InChI=1S/C19H19N7O6/c20-19-25-15-14(17(30)26-1...,[],N=c1nc(O)c2nc(CNc3ccc(C(=O)N[C@@H](CCC(=O)O)C(...,InChI=1S/C19H19N7O6/c20-19-25-15-14(17(30)26-1...,OVBPIULPVIDEAO-LBPRGKRZSA-N,OVBPIULPVIDEAO,C19H19N7O6,441.139681,0.19787,InChI=1S/C19H19N7O6/c20-19-25-15-14(17(30)26-1...,InChI=1S/C19H19N7O6/c20-19-25-15-14(17(30)26-1...
2,InChI=1S/C22H33NO/c1-3-4-5-6-7-8-9-10-11-12-15...,[],CCCCCCCCCCCCc1cc(=O)c2ccccc2n1C,InChI=1S/C22H33NO/c1-3-4-5-6-7-8-9-10-11-12-15...,URTNWZJTIJSNON-UHFFFAOYSA-N,URTNWZJTIJSNON,C22H33NO,327.256215,6.00190,InChI=1S/C22H33NO/c1-3-4-5-6-7-8-9-10-11-12-15...,InChI=1S/C22H33NO/c1-3-4-5-6-7-8-9-10-11-12-15...
3,InChI=1S/C9H13NO3/c1-10-5-9(13)6-2-3-7(11)8(12...,[],CNC[C@H](O)c1ccc(O)c(O)c1,InChI=1S/C9H13NO3/c1-10-5-9(13)6-2-3-7(11)8(12...,UCTWMZQNUQWSLP-VIFPVBQESA-N,UCTWMZQNUQWSLP,C9H13NO3,183.089543,0.35060,InChI=1S/C9H13NO3/c1-10-5-9(13)6-2-3-7(11)8(12...,InChI=1S/C9H13NO3/c1-10-5-9(13)6-2-3-7(11)8(12...
4,InChI=1S/C18H34O5/c1-2-3-4-8-11-16(20)17(21)14...,[],CCCCCCC(O)C(O)/C=C/C(O)CCCCCCC(=O)O,InChI=1S/C18H34O5/c1-2-3-4-8-11-16(20)17(21)14...,YFCZLXRIKFCQFU-BUHFOSPRSA-N,YFCZLXRIKFCQFU,C18H34O5,330.240624,3.02090,InChI=1S/C18H34O5/c1-2-3-4-8-11-16(20)17(21)14...,InChI=1S/C18H34O5/c1-2-3-4-8-11-16(20)17(21)14...


## Retrieve ClassyFire classifications

This first step is done using inchikey and interrogation of the gnps classified structures

In [9]:
gnps_proxy = True

In [10]:
url = "http://classyfire.wishartlab.com"
proxy_url =  "https://gnps-classyfire.ucsd.edu"
chunk_size = 1000
sleep_interval = 12

Below is a slightly modified version of the original get_classification() which will take list object as input

The while loop below allows to run the get_classifications() function until the number of classified inchikey stablizes. Indeed, it has been observed that when running the get_classifications() multiple times the number of classified structures was different. Since the requests are stocked in the .sqlite file implemenmting the while loop allows to fetch the maximum of structures by iterations.
The reason why the get_classifications() is not directly returning the maximal number of classified structures is unknown. It might be due to GET request errors which remain silent because of the parralelization of the function ?

In [11]:
all_inchi_keys = list(opendb['inchikey_sanitized'].drop_duplicates())

resolved_ik_number_list = [0, 0]
total_inchikey_number = len(all_inchi_keys)

while True:
    
    start_time = time.time()
    
    print('%s inchikey to resolve' % total_inchikey_number )
    get_classifications_cf_mod(all_inchi_keys)
    
    cleanse('all_json.json', 'all_json.json')
    
    with open("all_json.json") as tweetfile:
        jsondic = json.loads(tweetfile.read())

    df = json_normalize(jsondic)
    df = df.drop_duplicates( 'inchikey' )
    resolved_ik_number = len( df.drop_duplicates('inchikey').inchikey )
    resolved_ik_number_list.append( resolved_ik_number )
    print('%s resolved inchikeys' % resolved_ik_number )
    print("done in --- %s seconds ---" % (time.time() - start_time))
    
    if resolved_ik_number_list[-1] < resolved_ik_number_list[-2] or resolved_ik_number_list[-1] == resolved_ik_number_list[-3]:
        break

4253 inchikey to resolve
4224 resolved inchikeys
done in --- 32.12632703781128 seconds ---
4253 inchikey to resolve


KeyboardInterrupt: 

We then use the cleanse function to directly remove unclassified structures from the json. Else the json file is not treated by the json_normalize() function.
To remove null entries from json inputs and output cleaned file we define the cleanse() function. Slightly adapted from https://stackoverflow.com/a/50531943

In [ ]:
cleanse('all_json.json', 'all_json_cleaned.json')

We now load this cleaned json file

In [ ]:
with open("all_json_cleaned.json") as tweetfile:
        jsondic = json.loads(tweetfile.read())

And normalize the output as a dataframe

In [ ]:
flattened_classified_json = json_normalize(jsondic)

And have a peak into this new df

In [ ]:
flattened_classified_json.info()

In [ ]:
flattened_classified_json.drop_duplicates('inchikey').info()

We now want to output the unclassified IK

So first we strip the InChI= prefix of the previously returned df and keep it as a list

In [ ]:
+41 22 312 45 20classified_ik_list = list(flattened_classified_json['inchikey'].str.replace(r'InChIKey=', ''))

We now make the difference between the inputed ik list and the classified ik list

In [ ]:
len(set(all_inchi_keys))

In [ ]:
len(set(classified_ik_list))

In [ ]:
len(set(all_inchi_keys) - set(classified_ik_list))

## Merging the CF output with the original DB

In [ ]:
flattened_df = flattened_classified_json.drop_duplicates('inchikey')

In [ ]:
flattened_df['inchikey'] = flattened_df['inchikey'].str.replace(r'InChIKey=', '')

In [ ]:
df_merged = pd.merge(opendb,
                     flattened_df, 
                     left_on = 'inchikey_sanitized',
                     right_on = 'inchikey',
                     how = 'left')

In [ ]:
df_merged.info()

In [ ]:
df_merged_unclassed = df_merged[df_merged['inchikey'].isnull()]

In [ ]:
df_merged_unclassed.drop_duplicates('inchi_sanitized', 
                                    inplace = True)

In [ ]:
df_merged_unclassed.to_csv('test_datatable_unclassed.tsv',
                           sep = '\t', 
                           encoding = "utf-8")

## Classyfing the unclassified by inchi

Below is a modified version of the tabular_query() function which basically just launches a structure_query for all inchi or smile at a given column

In [ ]:
query_ids = batch_query('test_datatable_unclassed.tsv',
                        'inchi_sanitized',
                        dialect = 'excel-tab')

In [ ]:
query_ids

The status can also be checked manually at the following adress (just change the query id)
http://classyfire.wishartlab.com/queries/3879356.json?page=1

These settings of the request_cache allow to retry when 429 (or other) type of errors are returned by the classyfire server. Most of the time when too many intents are made. Since this seems to be a random behaviour, fixing a time.sleep is not safe enough.

In [ ]:
open("test_datatable_result_inchi.json", "w").write(json.dumps(Etienne-Dumont, 9get_results_multientry_multipage_patient(query_ids,
                                                                                                        return_format = "json")))

## Outputs standardization 

Now we will standardize the json output of classifire gent_entity() and the one of get_results_multipage_patient()

We will use the cleanse() function which as been previously defined to remove null entries from the json and normalize it.

### For round 1

For the output of the get_results_multipage_patient() we first load the json as a dataframe and remove identities with an empty identifier value

In [ ]:
with open("test_datatable_result_inchi.json") as tweetfile:
    jsondic_inchi = json.loads(tweetfile.read())

The json_normalize function is used to flatten the nested JSON structure.
Beware here the meta = ['id'] field can sometimes return an error. Remove if you dont need it.

In [ ]:
normalized_df_inchi = json_normalize(jsondic_inchi,
                                     record_path = 'entities',
                                     meta = ['label'])

In [ ]:
normalized_df_inchi.head()

And now we remove rows for wich no identifier is returned

In [ ]:
normalized_df_inchi_nona = normalized_df_inchi[normalized_df_inchi['identifier'].notna()]

Finally the previous df is outputed ad a json file using the 'records' option

In [ ]:
normalized_df_inchi_nona.to_json(r'result_json_all_cleaned.json', 
                                 orient = 'records')

## Merging both outputs

In [ ]:
flattened_classified_json.inchikey

In [ ]:
frames = [flattened_classified_json, normalized_df_inchi_nona]

result = pd.concat(frames)

In [ ]:
result.info()

In [ ]:
result.drop_duplicates('inchikey').info()

The resulting df can be exported as JSON file

In [ ]:
result.to_json(r'results_full.json',
               orient = 'records')

This file can be reloaded for further expansion of nested fields (to pursue later on)

In [ ]:
with open("results_full.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())

In [ ]:
normalized_df = json_normalize(jsondic,
                               record_path = 'intermediate_nodes',
                               meta = ['inchikey']
                              )

In [ ]:
 normalized_df = json_normalize(jsondic)

In [ ]:
normalized_df.head()

For now we will select fields of interest

First we load the total json and we normailze it as a df

In [ ]:
with open("results_full.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())
    normalized_df = json_normalize(jsondic)

We display all columns

In [ ]:
normalized_df.columns

We here make a list of unwanted coloumns name and we drop them

In [ ]:
colstodrop = ['smiles', 'intermediate_nodes', 'alternative_parents',
       'molecular_framework', 'substituents', 'description',
       'external_descriptors', 'ancestors', 'predicted_chebi_terms',
       'predicted_lipidmaps_terms', 'classification_version', 
       'kingdom.description', 'kingdom.chemont_id', 'kingdom.url',
        'superclass.description', 'superclass.chemont_id',
       'superclass.url',  'class.description', 'class.chemont_id',
       'class.url',  'subclass.description',
       'subclass.chemont_id', 'subclass.url', 
       'direct_parent.description', 'direct_parent.chemont_id',
       'direct_parent.url', 'identifier', 'subclass', 'label']

normalized_df.drop(colstodrop,
                   axis = 1,
                   inplace = True)

In [ ]:
normalized_df.info()

In [ ]:
normalized_df.head()

In [ ]:
normalized_df.drop_duplicates('inchikey').info()

Here we merge this classified df with the original df (after making sure bothe have been deduplicated and the InChIKey= string as been appended

In [ ]:
opendb.drop_duplicates('inchi_sanitized', 
                       inplace = True)

In [ ]:
normalized_df['inchikey'] = normalized_df['inchikey'].str.replace(r'InChIKey=', '')

In [ ]:
df_final_classy = pd.merge(opendb,
                           normalized_df, 
                           left_on = 'inchikey_sanitized', 
                           right_on = 'inchikey',
                           how = 'left')

In [ ]:
df_final_classy.drop_duplicates('inchikey_sanitized').info()

In [ ]:
df_final_classy.to_csv('final_classy.tsv', 
                       sep = '\t')